In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), rows AS (
        SELECT
            GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
            UNNEST(STRING_SPLIT(input, '\n')) row_content
        FROM input
    ), part_numbers AS (
        SELECT row_no, absolute_start col_start, absolute_end col_end, number_only part_no
        FROM (
            SELECT
                row_no,
                REGEXP_EXTRACT(number_text, '([0-9]+)') number_only,
                SUM(LENGTH(number_text)) OVER w absolute_end,
                LENGTH(number_text) relative_end,
                INSTR(number_text, number_only) relative_start,
                absolute_end - (relative_end - relative_start) absolute_start
            FROM (
                SELECT 
                    row_no,
                    row_content,
                    REGEXP_EXTRACT_ALL(row_content, '([^0-9]*[0-9]+)') numbers,
                    GENERATE_SUBSCRIPTS(numbers, 1) number_order,
                    UNNEST(numbers) number_text
                FROM rows
            )
            WINDOW w AS (PARTITION BY row_no ORDER BY number_order)
        )
    ), symbols AS (
        SELECT row_no, col_no, sbl
        FROM (
            SELECT
                row_no,
                GENERATE_SUBSCRIPTS(STRING_SPLIT(row_content, ''), 1) col_no,
                UNNEST(STRING_SPLIT(row_content, '')) sbl
            FROM rows
        )
        WHERE NOT sbl GLOB '[.0-9]'
    ), valid_part_numbers AS (
        SELECT CAST(part_no AS INT) part_no
        FROM part_numbers p
        WHERE EXISTS (
            SELECT *
            FROM symbols s
            WHERE s.row_no BETWEEN p.row_no - 1 AND p.row_no + 1
            AND s.col_no BETWEEN p.col_start - 1 AND p.col_end + 1
        )
    )
    SELECT SUM(part_no) ans
    FROM valid_part_numbers
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│ 550064 │
└────────┘

In [3]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), rows AS (
        SELECT
            GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
            UNNEST(STRING_SPLIT(input, '\n')) row_content
        FROM input
    ), part_numbers AS (
        SELECT row_no, absolute_start col_start, absolute_end col_end, number_only part_no
        FROM (
            SELECT
                row_no,
                REGEXP_EXTRACT(number_text, '([0-9]+)') number_only,
                SUM(LENGTH(number_text)) OVER w absolute_end,
                LENGTH(number_text) relative_end,
                INSTR(number_text, number_only) relative_start,
                absolute_end - (relative_end - relative_start) absolute_start
            FROM (
                SELECT 
                    row_no,
                    row_content,
                    REGEXP_EXTRACT_ALL(row_content, '([^0-9]*[0-9]+)') numbers,
                    GENERATE_SUBSCRIPTS(numbers, 1) number_order,
                    UNNEST(numbers) number_text
                FROM rows
            )
            WINDOW w AS (PARTITION BY row_no ORDER BY number_order)
        )
    ), symbols AS (
        SELECT row_no, col_no, sbl
        FROM (
            SELECT
                row_no,
                GENERATE_SUBSCRIPTS(STRING_SPLIT(row_content, ''), 1) col_no,
                UNNEST(STRING_SPLIT(row_content, '')) sbl
            FROM rows
        )
        WHERE NOT sbl GLOB '[.0-9]'
    ), gear_ratios AS (
        SELECT row_no, col_no, PRODUCT(CAST(part_no AS INT)) gr
        FROM (
            SELECT s.row_no, s.col_no, part_no
            FROM symbols s, part_numbers p
            WHERE s.sbl = '*'
            AND s.row_no BETWEEN p.row_no - 1 AND p.row_no + 1
            AND s.col_no BETWEEN p.col_start - 1 AND p.col_end + 1
        )
        GROUP BY row_no, col_no
        HAVING COUNT(*) = 2
    )
    SELECT CAST(SUM(gr) AS INT) ans
    FROM gear_ratios
""" % input)

┌──────────┐
│   ans    │
│  int32   │
├──────────┤
│ 85010461 │
└──────────┘